## MISR的TOA数据读取

### BRF

In [13]:
# for python 3.6
import os
from MisrToolkit import MtkFile, orbit_to_path, orbit_to_time_range, latlon_to_bls
import netCDF4
import numpy
import random
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.transforms as mtransforms
from scipy.stats import gaussian_kde, pearsonr
import math
import urllib.request
import ssl
import re

ssl._create_default_https_context = ssl._create_unverified_context
MISR_NC_FOLDER = '/disk1/Data/MISR4AHI2015070120210630_3'
MISR_TOA_FOLDER = '/disk1/Data/MISR4AHI2015070120210630_TOA'


def download_MISR_MIL2TCST02_HDF(folder, path, orbit):
    time_range = orbit_to_time_range(orbit)
    time0 = time_range[0]
    matchObj = re.search(r'(\d+)-(\d+)-(\d+)T', str(time0))
    yy = matchObj.group(1)
    mm = matchObj.group(2)
    dd = matchObj.group(3)
    t = str(yy) + '.' + str(mm) + '.' + str(dd)
    P = 'P' + (3 - len(str(path))) * '0' + str(path)
    O_ = 'O' + (6 - len(str(orbit))) * '0' + str(orbit)
    F = 'F' + '05'
    v = '0011'
    base_url = 'https://opendap.larc.nasa.gov/opendap/MISR/MIL2TCAL.002'
    filename = 'MISR_AM1_TC_ALBEDO_' + P + '_' + O_ + '_' + F + '_' + v + '.hdf'

    download_url = base_url + '/' + t + '/' + filename

    time1 = time_range[1]
    matchObj1 = re.search(r'(\d+)-(\d+)-(\d+)T', str(time1))
    yy1 = matchObj1.group(1)
    mm1 = matchObj1.group(2)
    dd1 = matchObj1.group(3)
    t1 = str(yy1) + '.' + str(mm1) + '.' + str(dd1)
    download_url1 = base_url + '/' + t1 + '/' + filename

    storage_path = folder + '/' + filename

    if os.path.exists(storage_path):
        try:
            m_file = MtkFile(storage_path)
            return storage_path
        except Exception as e:
            print(e)
            try:
                urllib.request.urlretrieve(download_url, filename=storage_path)
                return storage_path
            except Exception as e:
                print('Error: ' + download_url)
                print(e)
                try:
                    urllib.request.urlretrieve(download_url1, filename=storage_path)
                    return storage_path
                except Exception as e:
                    print('Error: ' + download_url1)
                    print(e)
                    return ''
    else:
        print('No file:', storage_path)
        try:
            urllib.request.urlretrieve(download_url, filename=storage_path)
            return storage_path
        except Exception as e:
            print('Error: ' + download_url)
            print(e)
            try:
                urllib.request.urlretrieve(download_url1, filename=storage_path)
                return storage_path
            except Exception as e:
                print('Error: ' + download_url1)
                print(e)
                return ''


def BRF_TrueValue(o_value, scale, offset):
    fill = 65533
    underflow = 65534
    overflow = 65535

    if o_value in [fill, underflow, overflow]:
        return 0.
    else:
        y = o_value * scale + offset
        return y
    
    
def record_roi_misr_ahi(lon, lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename):
    
    # print(roi_ahi_sr)
    misr_path = orbit_to_path(misr_orbit)
    # MISR v3 netCDF4
    misr_nc = netCDF4.Dataset(misr_nc_filename)
    misr_nc_11 = misr_nc.groups['1.1_KM_PRODUCTS']
    misr_brf_var = misr_nc_11.variables['Bidirectional_Reflectance_Factor']
    misr_brf_scalev3 = misr_brf_var.scale_factor
    misr_brf_offsetv3 = misr_brf_var.add_offset
    misr_nc.close()
    m_file2 = MtkFile(misr_nc_filename)
    m_grid11 = m_file2.grid('1.1_KM_PRODUCTS')
    misr_resolutionv3 = m_grid11.resolution
    m_field11 = m_grid11.field('Bidirectional_Reflectance_Factor[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')
    
    # MISR TOA HDF
    m_file = MtkFile(misr_hdf_filename)
    m_grid22 = m_file.grid('ReflectingLevelParameters_2.2_km')
    misr_resolution = m_grid22.resolution
    toa_field = m_grid22.field('BRFTop_Mean[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')  # band, camera

    # MISR data at ROI
    roi_misr_brfv3 = 0.
    roi_misr_toa = 0.
    
    try:
        misr_blsv3 = latlon_to_bls(misr_path, misr_resolutionv3, lat, lon)
        block_llv3 = misr_blsv3[0]
        b_lat_idxv3 = round(misr_blsv3[1])
        b_lon_idxv3 = round(misr_blsv3[2])

        misr_bls_toa = latlon_to_bls(misr_path, misr_resolution, lat, lon)
        block_ll_toa = misr_bls_toa[0]
        b_lat_idx_toa = round(misr_bls_toa[1])
        b_lon_idx_toa = round(misr_bls_toa[2])

        block_brf_dnv3 = m_field11.read(block_llv3, block_llv3)[0]
        roi_brf_dnv3 = block_brf_dnv3[b_lat_idxv3][b_lon_idxv3]
        roi_brf_tv3 = BRF_TrueValue(roi_brf_dnv3, misr_brf_scalev3, misr_brf_offsetv3)

        block_brf_dn_toa = toa_field.read(block_ll_toa, block_ll_toa)[0]
        roi_brf_t_toa = block_brf_dn_toa[b_lat_idx_toa][b_lon_idx_toa]

        roi_misr_brfv3 = roi_brf_tv3
        roi_misr_toa = roi_brf_t_toa
    except Exception as e:
        roi_misr_brfv3 = 0.
        roi_misr_toa = 0.

    # if any cloud-free obs. is existed
    if roi_misr_brfv3 > 0.:
        return roi_misr_brfv3, roi_misr_toa
    return 0., 0.


def get_roi_misr_ahi(roi_lon, roi_lat, misr_path_orbit_camera, band_index):
    misr_path_orbit = misr_path_orbit_camera[:12]
    misr_path = int(misr_path_orbit[1:4])
    misr_orbit = int(misr_path_orbit[-6:])
    misr_camera_index = int(misr_path_orbit_camera[-1:])
    
    misr_nc_filename = os.path.join(MISR_NC_FOLDER, 'MISR_AM1_AS_LAND_' + misr_path_orbit + '_F08_0023.nc')
    misr_hdf_filename = download_MISR_MIL2TCST02_HDF(MISR_TOA_FOLDER, misr_path, misr_orbit)

    roi_misr_toa = record_roi_misr_ahi(roi_lon, roi_lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename)
    return roi_misr_toa

In [58]:
roi_lon = 129.25
roi_lat = -20.25
misr_path_orbit_camera = 'P105_O093079_3'
band_index = 3
roi_misr_brf, roi_misr_toa = get_roi_misr_ahi(roi_lon, roi_lat, misr_path_orbit_camera, band_index)

In [59]:
roi_misr_brf # BRF

0.3272447

In [60]:
roi_misr_toa

0.3311854

### HDRF

In [29]:
def HDRF_TrueValue(o_value, scale, offset):
    fill = 65533
    underflow = 65534
    overflow = 65535

    if o_value in [fill, underflow, overflow]:
        return 0.
    else:
        y = o_value * scale + offset
        return y
    
    
def record_roi_misr_ahi_HDRF(lon, lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename):
    
    # print(roi_ahi_sr)
    misr_path = orbit_to_path(misr_orbit)
    # MISR v3 netCDF4
    misr_nc = netCDF4.Dataset(misr_nc_filename)
    misr_nc_11 = misr_nc.groups['1.1_KM_PRODUCTS']
    misr_brf_var = misr_nc_11.variables['Hemispherical_Directional_Reflectance_Factor']
    misr_brf_scalev3 = misr_brf_var.scale_factor
    misr_brf_offsetv3 = misr_brf_var.add_offset
    misr_nc.close()
    m_file2 = MtkFile(misr_nc_filename)
    m_grid11 = m_file2.grid('1.1_KM_PRODUCTS')
    misr_resolutionv3 = m_grid11.resolution
    m_field11 = m_grid11.field('Hemispherical_Directional_Reflectance_Factor[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')
    
    # MISR TOA HDF
    m_file = MtkFile(misr_hdf_filename)
    m_grid22 = m_file.grid('ReflectingLevelParameters_2.2_km')
    misr_resolution = m_grid22.resolution
    toa_field = m_grid22.field('BRFTop_Mean[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')  # band, camera

    # MISR data at ROI
    roi_misr_brfv3 = 0.
    roi_misr_toa = 0.
    
    try:
        misr_blsv3 = latlon_to_bls(misr_path, misr_resolutionv3, lat, lon)
        block_llv3 = misr_blsv3[0]
        b_lat_idxv3 = round(misr_blsv3[1])
        b_lon_idxv3 = round(misr_blsv3[2])

        misr_bls_toa = latlon_to_bls(misr_path, misr_resolution, lat, lon)
        block_ll_toa = misr_bls_toa[0]
        b_lat_idx_toa = round(misr_bls_toa[1])
        b_lon_idx_toa = round(misr_bls_toa[2])

        block_brf_dnv3 = m_field11.read(block_llv3, block_llv3)[0]
        roi_brf_dnv3 = block_brf_dnv3[b_lat_idxv3][b_lon_idxv3]
        roi_brf_tv3 = HDRF_TrueValue(roi_brf_dnv3, misr_brf_scalev3, misr_brf_offsetv3)

        block_brf_dn_toa = toa_field.read(block_ll_toa, block_ll_toa)[0]
        roi_brf_t_toa = block_brf_dn_toa[b_lat_idx_toa][b_lon_idx_toa]

        roi_misr_brfv3 = roi_brf_tv3
        roi_misr_toa = roi_brf_t_toa
    except Exception as e:
        roi_misr_brfv3 = 0.
        roi_misr_toa = 0.

    # if any cloud-free obs. is existed
    if roi_misr_brfv3 > 0.:
        return roi_misr_brfv3, roi_misr_toa
    return 0., 0.


def get_roi_misr_ahi_HDRF(roi_lon, roi_lat, misr_path_orbit_camera, band_index):
    misr_path_orbit = misr_path_orbit_camera[:12]
    misr_path = int(misr_path_orbit[1:4])
    misr_orbit = int(misr_path_orbit[-6:])
    misr_camera_index = int(misr_path_orbit_camera[-1:])
    
    misr_nc_filename = os.path.join(MISR_NC_FOLDER, 'MISR_AM1_AS_LAND_' + misr_path_orbit + '_F08_0023.nc')
    misr_hdf_filename = download_MISR_MIL2TCST02_HDF(MISR_TOA_FOLDER, misr_path, misr_orbit)

    roi_misr_toa = record_roi_misr_ahi_HDRF(roi_lon, roi_lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename)
    return roi_misr_toa

In [30]:
roi_lon = 129.25
roi_lat = -20.25
misr_path_orbit_camera = 'P105_O093079_3'
band_index = 3
roi_sr, roi_toa = get_roi_misr_ahi_HDRF(roi_lon, roi_lat, misr_path_orbit_camera, band_index)

In [31]:
roi_sr # HDRF

0.32693952

### BHR

In [42]:
def BHR_TrueValue(o_value, scale, offset):
    fill = 65533
    underflow = 65534
    overflow = 65535

    if o_value in [fill, underflow, overflow]:
        return 0.
    else:
        y = o_value * scale + offset
        return y
    
    
def record_roi_misr_ahi_BHR(lon, lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename):
    
    # print(roi_ahi_sr)
    misr_path = orbit_to_path(misr_orbit)
    # MISR v3 netCDF4
    misr_nc = netCDF4.Dataset(misr_nc_filename)
    misr_nc_11 = misr_nc.groups['1.1_KM_PRODUCTS']
    misr_brf_var = misr_nc_11.variables['Bi-Hemispherical_Reflectance']
    misr_brf_scalev3 = misr_brf_var.scale_factor
    misr_brf_offsetv3 = misr_brf_var.add_offset
    misr_nc.close()
    m_file2 = MtkFile(misr_nc_filename)
    m_grid11 = m_file2.grid('1.1_KM_PRODUCTS')
    misr_resolutionv3 = m_grid11.resolution
    m_field11 = m_grid11.field('Bi-Hemispherical_Reflectance[' + str(band_index) + ']')
    
    # MISR TOA HDF
    m_file = MtkFile(misr_hdf_filename)
    m_grid22 = m_file.grid('ReflectingLevelParameters_2.2_km')
    misr_resolution = m_grid22.resolution
    toa_field = m_grid22.field('BRFTop_Mean[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')  # band, camera

    # MISR data at ROI
    roi_misr_brfv3 = 0.
    roi_misr_toa = 0.
    
    try:
        misr_blsv3 = latlon_to_bls(misr_path, misr_resolutionv3, lat, lon)
        block_llv3 = misr_blsv3[0]
        b_lat_idxv3 = round(misr_blsv3[1])
        b_lon_idxv3 = round(misr_blsv3[2])

        misr_bls_toa = latlon_to_bls(misr_path, misr_resolution, lat, lon)
        block_ll_toa = misr_bls_toa[0]
        b_lat_idx_toa = round(misr_bls_toa[1])
        b_lon_idx_toa = round(misr_bls_toa[2])

        block_brf_dnv3 = m_field11.read(block_llv3, block_llv3)[0]
        roi_brf_dnv3 = block_brf_dnv3[b_lat_idxv3][b_lon_idxv3]
        roi_brf_tv3 = BHR_TrueValue(roi_brf_dnv3, misr_brf_scalev3, misr_brf_offsetv3)

        block_brf_dn_toa = toa_field.read(block_ll_toa, block_ll_toa)[0]
        roi_brf_t_toa = block_brf_dn_toa[b_lat_idx_toa][b_lon_idx_toa]

        roi_misr_brfv3 = roi_brf_tv3
        roi_misr_toa = roi_brf_t_toa
    except Exception as e:
        roi_misr_brfv3 = 0.
        roi_misr_toa = 0.

    # if any cloud-free obs. is existed
    if roi_misr_brfv3 > 0.:
        return roi_misr_brfv3, roi_misr_toa
    return 0., 0.


def get_roi_misr_ahi_BHR(roi_lon, roi_lat, misr_path_orbit_camera, band_index):
    misr_path_orbit = misr_path_orbit_camera[:12]
    misr_path = int(misr_path_orbit[1:4])
    misr_orbit = int(misr_path_orbit[-6:])
    misr_camera_index = int(misr_path_orbit_camera[-1:])
    
    misr_nc_filename = os.path.join(MISR_NC_FOLDER, 'MISR_AM1_AS_LAND_' + misr_path_orbit + '_F08_0023.nc')
    misr_hdf_filename = download_MISR_MIL2TCST02_HDF(MISR_TOA_FOLDER, misr_path, misr_orbit)

    roi_misr_toa = record_roi_misr_ahi_BHR(roi_lon, roi_lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename)
    return roi_misr_toa

In [43]:
roi_lon = 129.25
roi_lat = -20.25
misr_path_orbit_camera = 'P105_O093079_3'
band_index = 3
roi_sr, roi_toa = get_roi_misr_ahi_BHR(roi_lon, roi_lat, misr_path_orbit_camera, band_index)

In [44]:
roi_sr #BHR

0.31249997

### DHR

In [45]:
def DHR_TrueValue(o_value, scale, offset):
    fill = 65533
    underflow = 65534
    overflow = 65535

    if o_value in [fill, underflow, overflow]:
        return 0.
    else:
        y = o_value * scale + offset
        return y
    
    
def record_roi_misr_ahi_DHR(lon, lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename):
    
    # print(roi_ahi_sr)
    misr_path = orbit_to_path(misr_orbit)
    # MISR v3 netCDF4
    misr_nc = netCDF4.Dataset(misr_nc_filename)
    misr_nc_11 = misr_nc.groups['1.1_KM_PRODUCTS']
    misr_brf_var = misr_nc_11.variables['Directional_Hemispherical_Reflectance']
    misr_brf_scalev3 = misr_brf_var.scale_factor
    misr_brf_offsetv3 = misr_brf_var.add_offset
    misr_nc.close()
    m_file2 = MtkFile(misr_nc_filename)
    m_grid11 = m_file2.grid('1.1_KM_PRODUCTS')
    misr_resolutionv3 = m_grid11.resolution
    m_field11 = m_grid11.field('Directional_Hemispherical_Reflectance[' + str(band_index) + ']')
    
    # MISR TOA HDF
    m_file = MtkFile(misr_hdf_filename)
    m_grid22 = m_file.grid('ReflectingLevelParameters_2.2_km')
    misr_resolution = m_grid22.resolution
    toa_field = m_grid22.field('BRFTop_Mean[' + str(band_index) + ']' + '[' + str(misr_camera_index) + ']')  # band, camera

    # MISR data at ROI
    roi_misr_brfv3 = 0.
    roi_misr_toa = 0.
    
    try:
        misr_blsv3 = latlon_to_bls(misr_path, misr_resolutionv3, lat, lon)
        block_llv3 = misr_blsv3[0]
        b_lat_idxv3 = round(misr_blsv3[1])
        b_lon_idxv3 = round(misr_blsv3[2])

        misr_bls_toa = latlon_to_bls(misr_path, misr_resolution, lat, lon)
        block_ll_toa = misr_bls_toa[0]
        b_lat_idx_toa = round(misr_bls_toa[1])
        b_lon_idx_toa = round(misr_bls_toa[2])

        block_brf_dnv3 = m_field11.read(block_llv3, block_llv3)[0]
        roi_brf_dnv3 = block_brf_dnv3[b_lat_idxv3][b_lon_idxv3]
        roi_brf_tv3 = DHR_TrueValue(roi_brf_dnv3, misr_brf_scalev3, misr_brf_offsetv3)

        block_brf_dn_toa = toa_field.read(block_ll_toa, block_ll_toa)[0]
        roi_brf_t_toa = block_brf_dn_toa[b_lat_idx_toa][b_lon_idx_toa]

        roi_misr_brfv3 = roi_brf_tv3
        roi_misr_toa = roi_brf_t_toa
    except Exception as e:
        roi_misr_brfv3 = 0.
        roi_misr_toa = 0.

    # if any cloud-free obs. is existed
    if roi_misr_brfv3 > 0.:
        return roi_misr_brfv3, roi_misr_toa
    return 0., 0.


def get_roi_misr_ahi_DHR(roi_lon, roi_lat, misr_path_orbit_camera, band_index):
    misr_path_orbit = misr_path_orbit_camera[:12]
    misr_path = int(misr_path_orbit[1:4])
    misr_orbit = int(misr_path_orbit[-6:])
    misr_camera_index = int(misr_path_orbit_camera[-1:])
    
    misr_nc_filename = os.path.join(MISR_NC_FOLDER, 'MISR_AM1_AS_LAND_' + misr_path_orbit + '_F08_0023.nc')
    misr_hdf_filename = download_MISR_MIL2TCST02_HDF(MISR_TOA_FOLDER, misr_path, misr_orbit)

    roi_misr_toa = record_roi_misr_ahi_DHR(roi_lon, roi_lat, band_index, misr_orbit, misr_camera_index, misr_nc_filename, misr_hdf_filename)
    return roi_misr_toa

In [46]:
roi_lon = 129.25
roi_lat = -20.25
misr_path_orbit_camera = 'P105_O093079_3'
band_index = 3
roi_sr, roi_toa = get_roi_misr_ahi_DHR(roi_lon, roi_lat, misr_path_orbit_camera, band_index)

In [47]:
roi_sr # DHR

0.31249997

## AHI的TOA数据读取

In [11]:
DN2Tbb_folder = '/disk1/Data/band_DN2Tbb'
BAND_INFO = {
    'band1': ['VIS', '0.01', '.vis.01.fld.geoss.bz2', 'vis.01'],
    'band2': ['VIS', '0.01', '.vis.02.fld.geoss.bz2', 'vis.02'],
    'band3': ['EXT', '0.005', '.ext.01.fld.geoss.bz2', 'ext.01'],
    'band4': ['VIS', '0.01', '.vis.03.fld.geoss.bz2', 'vis.03'],
}


def roi_ahi_data_dn(r_lon, r_lat, ahi_bin, o_resolution):
    # get roi data array with AHI resolution (1km)
    lons = numpy.arange(85. + o_resolution / 2, 205, o_resolution)
    lats = numpy.arange(60. - o_resolution / 2, -60, -o_resolution)
    ahi_dn = numpy.fromfile(ahi_bin, dtype='>u2')
    ahi_dn = ahi_dn.reshape(len(lats), len(lons))
    
    data_ahi_roi = ahi_dn[int((60-r_lat)/o_resolution)][int((r_lon-85)/o_resolution)]
    
    return data_ahi_roi


def roi_ahi_data(r_lon, r_lat, filename, band_jma):
    band_info_ = BAND_INFO[band_jma]
    resolution = float(band_info_[1])
    roi_ahi_dn = roi_ahi_data_dn(r_lon, r_lat, filename, resolution)
    # DN to Tbb
    LUT_file = band_info_[3]
    LUT_filename = os.path.join(DN2Tbb_folder, LUT_file)
    DN2Tbb_LUT = numpy.loadtxt(LUT_filename)
    roi_ahi_ref = numpy.zeros_like(roi_ahi_dn)
    roi_ahi_ref = DN2Tbb_LUT[int(roi_ahi_dn), 1]
    roi_ahi_ref = roi_ahi_ref / 100
    return roi_ahi_ref

In [56]:
roi_lon = 129.25
roi_lat = -20.25
ahi_obstime = 201706180130
ahi_filename = '/disk1/workspace/20230615/201706180130.vis.03.fld.geoss'
roi_ahi_toa = roi_ahi_data(roi_lon, roi_lat, ahi_filename, 'band4')

In [50]:
roi_lon = 129.25
roi_lat = -20.25
ahi_filename = '/disk1/workspace/20230615/201706180130.sun.zth.fld.4km.bin'
# get roi angle array with AHI resolution (4km)
a_resolution = 0.04
lons = numpy.arange(85. + a_resolution / 2, 205, a_resolution)
lats = numpy.arange(60. - a_resolution / 2, -60, -a_resolution)
ahi_dn = numpy.fromfile(ahi_filename, dtype='>f4')
ahi_dn = ahi_dn.reshape(len(lats), len(lons))
sza_ahi_roi = ahi_dn[int((60-roi_lat)/a_resolution)][int((roi_lon-85)/a_resolution)]

In [62]:
roi_ahi_toa

0.20411764999999998

In [51]:
sza_ahi_roi # degree

50.79307

In [63]:
roi_misr_toa

0.3311854

In [64]:
roi_misr_toa * math.cos(math.radians(sza_ahi_roi))

0.20934991055876837